In [3]:
## Motif Marker:
## Adrian Bubie
## 2/11/18
## --------------------
## This program detects motifs (selected sequences) of interest from sequences provided in a fasta format. 
## As the intended use case is to identify motifs around exon boundries that are potential regulators for splicing events, 
## this program maps detected motifs onto a representation of the intron/exons sequences provided in the input file, and
## exports these maps as an SVG graphic.

import textwrap
import random
import re
import argparse as ap
import cairo as pc
import math

In [ ]:
def get_arguments():
    parser = ap.ArgumentParser(prog='./Motif_Marker.py', formatter_class=ap.RawDescriptionHelpFormatter, description=textwrap.dedent('''\
    Motif Sequence Marker
    ---------------------
    Locates and counts specified motif sequences from given gene sequences
    in FASTA format, then maps motifs' locations in relation to Intron/Exon of given sequence.
    
    Takes input of FASTA file with any exonic sequences capitalized, and intronic sequences lowercase.
    Requires motifs to search for and size of intronic sequence flanking exons to be searched to be specified by the user. 
    '''))
    parser.add_argument("-f", help="Fasta file to be processed. Exon sequences must be capitalized, and gene names contained in sequence ID. Must include absolute path the file. <str>", required=True, type=str)
    parser.add_argument("-w", help="Intronic flanking sequence window sized (applies to both sides of exonic regions). Default is 200bp. <int>", required=False, type=int, default=200)
    parser.add_argument("-m", help="Motif file to define motif sequences to query for (one Motif per line). Motifs must use *IUPAC* Nucleotide codes. Must include absolute path to file <str>", required=True, type=str)
    parser.add_argument("-counts", help="Optional output file containing the count of each motif per fasta sequence. File is created in location of imput file. <str>", required=False, type=str, default='')
    parser.add_argument("-colors", help="Set color palate to use for motif markers (note: whole sequences will always be represented in black). Provide as list of hex codes (#)", required=False, type=str, default='')
    return parser.parse_args()

In [4]:
def iupac_interp(motifs_file):
    '''IUPAC Interpreter: Takes motif file, parses out motifs, and returns list of regex search terms for each motif
       based on IUPAC nomencalture.'''
    
    # Keys of dict are nucleotide code, and values are bases:
    iupac = {"A":'[Aa]',"C":'[Cc]',"G":'[Gg]',"T":'[TUtu]',"U":'[TUtu]',"R":'[AGag]',"Y":'[CTct]',"S":'[GCgc]',"W":'[ATat]',"K":'[GTgt]',"M":'[ACac]',"B":'[CGTcgt]',"D":'[AGTagt]',"H":'[ACTact]',"V":'[ACGacg]',"N":'[A-Za-z]',
             "a":'[Aa]',"c":'[Cc]',"g":'[Gg]',"t":'[TUtu]',"u":'[TUtu]',"r":'[AGag]',"y":'[CTct]',"s":'[GCgc]',"w":'[ATat]',"k":'[GTgt]',"m":'[ACac]',"b":'[CGTcgt]',"d":'[AGTagt]',"h":'[ACTact]',"v":'[ACGacg]',"n":'[A-Za-z]'}
    
    with open(motifs_file, 'r') as mtfs:
        search_terms = []
        line = mtfs.readline()
        while line:
            st = ''
            mt = str(line).strip('\n')
            for char in mt:
                if char in iupac.keys():
                    st = st+iupac[char]
                else:
                    raise ValueError('Error: motif contains character not in IUPAC nucleotide codes; motif cannot be translated') # Throw exception if the motif contains character not in IUPAC
            
            search_terms.append(st)
            line = mtfs.readline()
    
    return search_terms
                

In [6]:
iupac_interp('/Users/Adrian/BGMP/Motif_marker/test/motifs.txt')

['[CTct][Gg][Cc][CTct]',
 '[TUtu][Gg][Cc][Aa][TUtu][Gg]',
 '[Gg][Cc][Aa][TUtu][Gg]']

In [33]:
class fasta_sequence():
    '''Fasta Sequence object: collection of sequence lines for a single fasta ID.
       Object collects lines into single string for motif searching, collects seq length, and more.'''
    
    def __init__(self, lines):
        self.gene = lines[0].split(' ')[0].strip('>')
        self.seq = ''.join([line.strip('\n') for line in lines if line.startswith('>') != True])
    
    def tot_seq_len(self):
        return int(len(self.seq))
        

In [34]:
m = fasta_sequence(['>IRSN chr:12342\n','ATCGCTACGT\n','TCGCTacgtcgTC\n','ATCGTACTAGC\n'])

In [38]:
####################
## Main function: ##
####################

Fasta = open('/Users/Adrian/BGMP/Motif_marker/test/fasta_t1.fa','r')
line = Fasta.readline()
#seqs = []

while line:
    to_process = []
    to_process.append(line)
    line = Fasta.readline()
    while (line.startswith('>') != True) and line != '':
        to_process.append(line)
        line = Fasta.readline()
    
    #seqs.append(to_process)

Fasta.close()